## Librerias a utilizar

In [44]:
import pandas as pd
import numpy as np
from itertools import combinations

Lectura de la base

In [47]:
base = pd.read_csv(".\\BI.csv",dtype = {'ClientID':'str'})  #leer la base
#base.dtypes

##Crear tres variables
#Movimiento todos igual a 1
#In 1 si el movimiento fue de alta, 0 de otra manera
#Out 1 si el movimiento fue de baja, 0 de otra manera
base = base.assign(MOVIMIENTO = 1) 
base['IN'] = np.where(base.MOVEMENT=='in', 1,0)
base['OUT'] = np.where(base.MOVEMENT == 'out', 1, 0)
base = base[base.ClientID != '442030']

In [48]:
base_agrupada = base.groupby('ClientID') #agrupas baser por cliente
movimientos_cliente = base_agrupada[['MOVIMIENTO','IN', 'OUT']].sum() #calcular el numero de movimientos, altas y bajas hechas por cada cliente
movimientos_cliente.reset_index()                                       #resetear el indice
movimientos_cliente.sort_values(['MOVIMIENTO'], ascending=False, inplace= True)#ordenar de mayor a menor
movimientos_cliente = movimientos_cliente[movimientos_cliente.MOVIMIENTO>2]#filtrar por clientes con 3 o más movimientos
movimientos_cliente['ClientID'] = movimientos_cliente.index#crear la variable ClienteID a partir del indice
clientes = movimientos_cliente.ClientID #guardar los clientes con mas de 2 movimientos
clientes = list(clientes) #guardar como lista a los clientes con más de 2 movimientos

In [49]:
base_filtrada = base[base.ClientID.isin(clientes)] #filtar la base original por los clientes con más de 2 movimientos

#Clientes_unicos = base_filtrada.ClientID.unique()
#base_filtrada.ClientID.value_counts()

In [50]:
aristas = []
for cliente in clientes:
    base_filtrada2 = base_filtrada[base_filtrada.ClientID==cliente]
    #print(cliente)
    #print(len(base_filtrada2))
    lista_agentid = base_filtrada2['AGENT ID']
    lista_agentid =list(set(lista_agentid))
    #print(lista_agentid)
    lista_combinaciones = list(combinations(lista_agentid,2))
    for elemento in lista_combinaciones:
        aristas.append(elemento)



In [51]:
len(aristas)

1571

In [52]:
aristas2=[]
for arista in aristas:
    arista_reverse = (arista[1], arista[0])
    if arista in aristas2:
        aristas2.append(arista)
    elif arista_reverse in aristas2:
        aristas2.append(arista_reverse)
    else:
        aristas2.append(arista)
#aristas2


In [53]:
diccionario = {}
for arista in aristas2:
    if arista in diccionario:
        diccionario[arista]+=1
    else:
        diccionario[arista] = 1

In [54]:
diccionario

{(11624, 22369): 1,
 (11624, 22380): 2,
 (22369, 22380): 1,
 (20728, 29054): 2,
 (20728, 6510): 1,
 (29054, 6510): 3,
 (13028, 23623): 2,
 (26434, 13827): 1,
 (26434, 24804): 1,
 (13827, 24804): 1,
 (24828, 1237): 1,
 (6006, 6527): 5,
 (16240, 11026): 1,
 (23755, 23623): 1,
 (17700, 30621): 8,
 (17700, 26799): 3,
 (30621, 26799): 4,
 (22290, 25252): 1,
 (22290, 30292): 2,
 (25252, 30292): 1,
 (16240, 24377): 3,
 (21856, 16483): 8,
 (21856, 7004): 7,
 (16483, 7004): 5,
 (24476, 24095): 1,
 (23755, 6510): 1,
 (8556, 18356): 2,
 (8556, 13148): 1,
 (18356, 13148): 1,
 (20920, 3375): 1,
 (8556, 9326): 2,
 (18700, 13276): 1,
 (18700, 10268): 1,
 (13276, 10268): 1,
 (28788, 16558): 1,
 (6098, 20564): 4,
 (16602, 30621): 2,
 (16602, 26799): 1,
 (14842, 18356): 1,
 (22408, 22231): 2,
 (14842, 16555): 1,
 (14768, 18030): 1,
 (14768, 30439): 1,
 (18030, 30439): 1,
 (20920, 9915): 1,
 (16032, 10678): 2,
 (23944, 8410): 1,
 (23944, 30621): 4,
 (8410, 30621): 8,
 (8464, 24555): 1,
 (24555, 16558): 1

In [55]:
grafo = pd.DataFrame(columns=['Src','aux', 'Dest','Weight'])
grafo

,Src,aux,Dest,Weight


In [56]:
for entrada in diccionario:
    #print(entrada[0],entrada[1],":",diccionario[entrada])
    grafo = grafo.append({'Src':'Agente' + str(entrada[0]),'aux':1, 'Dest': 'Agente' + str(entrada[1]), 'Weight':diccionario[entrada]},ignore_index=True)
    #grafo = grafo.append({'Dest':'Agente' + str(entrada[0]),'aux':1, 'Src': 'Agente' + str(entrada[1]), 'Weight':diccionario[entrada]},ignore_index=True)
    

In [58]:
grafo.sort_values('Weight', ascending=False)

,Src,aux,Dest,Weight
84,Agente25272,1,Agente16558,14
225,Agente6098,1,Agente21662,9
211,Agente23944,1,Agente17700,9
14,Agente17700,1,Agente30621,8
48,Agente8410,1,Agente30621,8
...,...,...,...,...
488,Agente24589,1,Agente28245,1
487,Agente28106,1,Agente11082,1
486,Agente2433,1,Agente22935,1
484,Agente25275,1,Agente22805,1


In [59]:
grafo.to_csv("grafo.csv",index=False)